In [1]:
from __future__ import annotations

import re
import unicodedata

from pathlib import Path

import numpy as np
import pandas as pd


# =============================================================================
# Helpers: normalização de texto
# =============================================================================
def _norm_text(x: object) -> str:
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return ''
    s = str(x).strip().lower()
    s = unicodedata.normalize('NFKD', s)
    s = ''.join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r'\s+', ' ', s)
    return s


# =============================================================================
# Parsers (inspirados no SQL legado)
# =============================================================================
def parse_concentration_to_g_per_unit(conc_raw: object) -> float:
    """
    Converte concentração para 'gramas por unidade' (proxy).
    Exemplos aceitos:
      - '500 mg' -> 0.5 g
      - '1 g' -> 1.0 g
      - '250mg/5ml' -> 0.05 g por ml (250mg/5ml => 50mg/ml => 0.05g/ml)
    Retorna np.nan quando não conseguir inferir.
    """
    s = _norm_text(conc_raw)
    if not s:
        return np.nan

    # mg/ml no formato 250mg/5ml
    m = re.search(r'(\d+(\.\d+)?)\s*mg\s*/\s*(\d+(\.\d+)?)\s*ml', s)
    if m:
        mg = float(m.group(1))
        ml = float(m.group(3))
        return (mg / 1000.0) / ml if ml > 0 else np.nan  # g por ml

    # mg/ml no formato 50 mg/ml
    m = re.search(r'(\d+(\.\d+)?)\s*mg\s*/\s*ml', s)
    if m:
        return float(m.group(1)) / 1000.0  # g por ml

    # mg simples
    m = re.search(r'(\d+(\.\d+)?)\s*mg', s)
    if m:
        return float(m.group(1)) / 1000.0

    # g simples
    m = re.search(r'(\d+(\.\d+)?)\s*g', s)
    if m:
        return float(m.group(1))

    # UI sem conversão
    if 'ui' in s:
        return np.nan

    return np.nan


def parse_days_of_treatment(dur_raw: object) -> float:
    """
    Extrai número de dias de tratamento a partir de texto ou numérico.
    Exemplos:
      - '7 dias' -> 7
      - 'por 10 dias' -> 10
      - '2 semanas' -> 14
      - 'dose unica' -> 1
    """
    if dur_raw is None or (isinstance(dur_raw, float) and np.isnan(dur_raw)):
        return np.nan

    if isinstance(dur_raw, (int, np.integer)):
        return float(dur_raw)
    if isinstance(dur_raw, (float, np.floating)) and not np.isnan(dur_raw):
        return float(dur_raw)

    s = _norm_text(dur_raw)

    # dose única
    if any(k in s for k in ['dose unica', 'dose única', 'unica dose', 'uma unica vez', 'apenas uma vez', 'agora']):
        return 1.0

    # semanas
    m = re.search(r'(\d+)\s*seman', s)
    if m:
        return float(m.group(1)) * 7.0

    # dias
    m = re.search(r'(\d+)\s*dia', s)
    if m:
        return float(m.group(1))

    # horas
    m = re.search(r'(\d+)\s*h', s)
    if m:
        return float(m.group(1)) / 24.0

    return np.nan


def parse_times_per_day(dur_raw: object) -> float:
    """
    Inferência de frequência diária a partir do texto.
    Exemplos:
      - 'a cada 8h' -> 3
      - 'a cada 12h' -> 2
      - 'de 8/8h' -> 3
      - '2x ao dia' -> 2
      - dose única -> 1
    """
    s = _norm_text(dur_raw)
    if not s:
        return np.nan

    if any(k in s for k in ['dose unica', 'dose única', 'unica dose', 'uma unica vez', 'apenas uma vez', 'agora']):
        return 1.0

    # a cada Xh
    m = re.search(r'a cada\s*(\d+)\s*h', s)
    if m:
        h = int(m.group(1))
        return float(round(24 / h)) if h > 0 else np.nan

    # 8/8h, 12/12h etc.
    m = re.search(r'(\d+)\s*/\s*(\d+)\s*h', s)
    if m:
        h = int(m.group(2))
        return float(round(24 / h)) if h > 0 else np.nan

    # Nx ao dia
    m = re.search(r'(\d+)\s*x\s*ao dia', s)
    if m:
        return float(m.group(1))

    # textos comuns
    if 'duas vezes ao dia' in s:
        return 2.0
    if 'tres vezes ao dia' in s:
        return 3.0
    if 'quatro vezes ao dia' in s:
        return 4.0
    if 'uma vez ao dia' in s:
        return 1.0

    return np.nan


def parse_qty_per_dose(dur_raw: object) -> float:
    """
    Inferência de quantidade por dose (comprimidos, cápsulas, ml, ampolas).
    Default conservador: 1.
    """
    s = _norm_text(dur_raw)
    if not s:
        return 1.0

    m = re.search(r'(\d+(\.\d+)?)\s*(cp|comprim|comp|caps|capsula|cápsula|drg|dragea)', s)
    if m:
        return float(m.group(1))

    m = re.search(r'(\d+(\.\d+)?)\s*ml', s)
    if m:
        return float(m.group(1))

    m = re.search(r'(\d+(\.\d+)?)\s*(amp|ampola)', s)
    if m:
        return float(m.group(1))

    return 1.0


# =============================================================================
# Função principal: recebe DF GOLD flat e retorna com features novas
# =============================================================================
def add_ddd_proxy_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Recebe um DF com as colunas do Gold flat (incluindo concentracao e duracao)
    e adiciona features de intensidade terapêutica (proxy para DDD).

    Espera pelo menos:
      - 'concentracao'
      - 'duracao'

    Retorna cópia com novas colunas:
      - 'dias_tratamento'
      - 'vezes_ao_dia'
      - 'qtd_por_dose'
      - 'conc_g_por_unid'
      - 'dose_total_pres_g'
      - 'e_dose_unica'
    """
    required = {'concentracao', 'duracao'}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f'Colunas ausentes no df: {sorted(missing)}')

    out = df.copy()

    # normalização para debug/auditoria
    out['duracao_norm'] = out['duracao'].map(_norm_text)

    out['dias_tratamento'] = out['duracao'].map(parse_days_of_treatment)
    out['vezes_ao_dia'] = out['duracao'].map(parse_times_per_day)
    out['qtd_por_dose'] = out['duracao'].map(parse_qty_per_dose)

    out['conc_g_por_unid'] = out['concentracao'].map(parse_concentration_to_g_per_unit)

    # default conservador para frequência quando ausente
    out['vezes_ao_dia'] = out['vezes_ao_dia'].fillna(1.0)

    # dose única
    out['e_dose_unica'] = out['dias_tratamento'].fillna(np.nan).le(1)

    # proxy: gramas totais prescritas
    out['dose_total_pres_g'] = (
        out['conc_g_por_unid']
        * out['qtd_por_dose']
        * out['vezes_ao_dia']
        * out['dias_tratamento']
    )

    return out

In [2]:
gold_path   = Path('../../data/gold/')
df  = pd.read_parquet(gold_path / 'full_data.parquet')
df

,cod_atendimento,cod_paciente,data_atendimento,sexo,idade,especialidade,cod_cid_ciap,diagnosticar_por,diag_agrupado,diag_analise,...,faixa_etaria,cod_medicamento,nome_medicamento,composto_quimico,concentracao,unidade_apresentacao,duracao,e_antibiotico,n_medicacao,e_presc_inadequada
0,0006ac77f5995f1fad05472a64145d987d529443,67044,2023-07-03,M,74,MÉDICO CLÍNICO 225125,B07,1,Não Especificado,Não Especificado,...,60+ anos,55448,DIPIRONA SODICA 500MG,DIPYRONE,500MG,COMPRIMIDO,Até 01 comprimido até de 6/6 h se dor (2 SE D...,False,3,True
1,0006ac77f5995f1fad05472a64145d987d529443,67044,2023-07-03,M,74,MÉDICO CLÍNICO 225125,B07,1,Não Especificado,Não Especificado,...,60+ anos,57558,IBUPROFENO 600MG,IBUPROFEN,600MG,COMPRIMIDO,"TOMAR 01 COMP, VIA ORAL, DE 8/8H, POR 5 DIAS",False,3,True
2,0006ac77f5995f1fad05472a64145d987d529443,67044,2023-07-03,M,74,MÉDICO CLÍNICO 225125,B07,1,Não Especificado,Não Especificado,...,60+ anos,775364,GAZE E INSUMOS PARA CURATIVO,Não Especificado,.,PACOTE,APLICAR SOBRE O LOCAL.,False,3,True
3,000756d289cac1ed88c15375bb9b6929abb0fa50,7358,2023-06-14,M,68,MÉDICO CLÍNICO 225125,B029,1,Não Especificado,Não Especificado,...,60+ anos,57581,"TRAMADOL, CL. 50MG",TRAMADOL,50MG,CAPSULA,USO ORAL\n\nTOMAR 1 CP DE 8/8H SE DOR FORTE,False,2,True
4,000756d289cac1ed88c15375bb9b6929abb0fa50,7358,2023-06-14,M,68,MÉDICO CLÍNICO 225125,B029,1,Não Especificado,Não Especificado,...,60+ anos,57709,ACICLOVIR 200MG CP,Não Especificado,200MG,COMPRIMIDO,02 CP DE 4/4H POR 7 DIAS (5 TOMADAS AO DIA),False,2,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470531,fffba8e8fec51f8d028e5c8570dbb9b5d21bd73a,73989,2023-08-28,F,78,MÉDICO DA ESTRATÉGIA DE SAÚDE DA FAMÍLIA 225142,Z000,1,General examination and observation of newborn,Não Especificado,...,60+ anos,57874,SINVASTATINA 10MG,SIMVASTATIN,10MG,COMPRIMIDO,Tomar 1 cp à noite.,False,6,False
470532,fffba8e8fec51f8d028e5c8570dbb9b5d21bd73a,73989,2023-08-28,F,78,MÉDICO DA ESTRATÉGIA DE SAÚDE DA FAMÍLIA 225142,Z000,1,General examination and observation of newborn,Não Especificado,...,60+ anos,388264,ADDERA D3,Não Especificado,1000 UI,COMPRIMIDO,Tomar 1 cp ao dia.,False,6,False
470533,fffba8e8fec51f8d028e5c8570dbb9b5d21bd73a,73989,2023-08-28,F,78,MÉDICO DA ESTRATÉGIA DE SAÚDE DA FAMÍLIA 225142,Z000,1,General examination and observation of newborn,Não Especificado,...,60+ anos,388267,MODURETIC,Não Especificado,25 mg,COMPRIMIDO,Tomar 1/2 cp de manhã,False,6,False
470534,fffda6e17014d4495ca72b1a3b22db7772f6464a,47367,2023-03-22,M,19,MÉDICO CLÍNICO 225125,R05,1,Cough,Não Especificado,...,18-59 anos,57640,LORATADINA 10MG,LORATADINE,10MG,COMPRIMIDO,TOMAR 1 CP VO A NOITE POR 5 DIAS.,False,1,False


In [3]:
df = add_ddd_proxy_features(df)
df

,cod_atendimento,cod_paciente,data_atendimento,sexo,idade,especialidade,cod_cid_ciap,diagnosticar_por,diag_agrupado,diag_analise,...,e_antibiotico,n_medicacao,e_presc_inadequada,duracao_norm,dias_tratamento,vezes_ao_dia,qtd_por_dose,conc_g_por_unid,e_dose_unica,dose_total_pres_g
0,0006ac77f5995f1fad05472a64145d987d529443,67044,2023-07-03,M,74,MÉDICO CLÍNICO 225125,B07,1,Não Especificado,Não Especificado,...,False,3,True,ate 01 comprimido ate de 6/6 h se dor (2 se do...,0.250000,4.0,1.0,0.500,True,0.50
1,0006ac77f5995f1fad05472a64145d987d529443,67044,2023-07-03,M,74,MÉDICO CLÍNICO 225125,B07,1,Não Especificado,Não Especificado,...,False,3,True,"tomar 01 comp, via oral, de 8/8h, por 5 dias",5.000000,3.0,1.0,0.600,False,9.00
2,0006ac77f5995f1fad05472a64145d987d529443,67044,2023-07-03,M,74,MÉDICO CLÍNICO 225125,B07,1,Não Especificado,Não Especificado,...,False,3,True,aplicar sobre o local.,NaN,1.0,1.0,NaN,False,NaN
3,000756d289cac1ed88c15375bb9b6929abb0fa50,7358,2023-06-14,M,68,MÉDICO CLÍNICO 225125,B029,1,Não Especificado,Não Especificado,...,False,2,True,uso oral tomar 1 cp de 8/8h se dor forte,0.333333,3.0,1.0,0.050,True,0.05
4,000756d289cac1ed88c15375bb9b6929abb0fa50,7358,2023-06-14,M,68,MÉDICO CLÍNICO 225125,B029,1,Não Especificado,Não Especificado,...,False,2,True,02 cp de 4/4h por 7 dias (5 tomadas ao dia),7.000000,6.0,2.0,0.200,False,16.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470531,fffba8e8fec51f8d028e5c8570dbb9b5d21bd73a,73989,2023-08-28,F,78,MÉDICO DA ESTRATÉGIA DE SAÚDE DA FAMÍLIA 225142,Z000,1,General examination and observation of newborn,Não Especificado,...,False,6,False,tomar 1 cp a noite.,NaN,1.0,1.0,0.010,False,NaN
470532,fffba8e8fec51f8d028e5c8570dbb9b5d21bd73a,73989,2023-08-28,F,78,MÉDICO DA ESTRATÉGIA DE SAÚDE DA FAMÍLIA 225142,Z000,1,General examination and observation of newborn,Não Especificado,...,False,6,False,tomar 1 cp ao dia.,NaN,1.0,1.0,NaN,False,NaN
470533,fffba8e8fec51f8d028e5c8570dbb9b5d21bd73a,73989,2023-08-28,F,78,MÉDICO DA ESTRATÉGIA DE SAÚDE DA FAMÍLIA 225142,Z000,1,General examination and observation of newborn,Não Especificado,...,False,6,False,tomar 1/2 cp de manha,NaN,1.0,2.0,0.025,False,NaN
470534,fffda6e17014d4495ca72b1a3b22db7772f6464a,47367,2023-03-22,M,19,MÉDICO CLÍNICO 225125,R05,1,Cough,Não Especificado,...,False,1,False,tomar 1 cp vo a noite por 5 dias.,5.000000,1.0,1.0,0.010,False,0.05


In [4]:
def build_attendance_level_df(
    df: pd.DataFrame,
    ensure_proxy_features: bool = True,
) -> pd.DataFrame:
    '''
    Constrói um DataFrame no nível de atendimento (cod_atendimento) a partir do Gold flat.

    Espera colunas do schema:
      - 'cod_atendimento', 'cod_paciente', 'data_atendimento', 'sexo', 'idade', 'especialidade',
        'cod_cid_ciap', 'diagnosticar_por', 'diag_agrupado', 'diag_analise', 'e_diag_infeccioso',
        'ano', 'mes', 'nome_mes', 'ano_mes', 'cod_unidade_saude', 'nome_unidade', 'tipo', 'e_analizada',
        'n_cid_ciap', 'faixa_etaria', 'cod_medicamento', 'nome_medicamento', 'composto_quimico',
        'concentracao', 'unidade_apresentacao', 'duracao', 'e_antibiotico', 'n_medicacao',
        'e_presc_inadequada'

    Se ensure_proxy_features=True e as colunas de features não existirem,
    chama add_ddd_proxy_features(df) para criar:
      - 'dias_tratamento', 'e_dose_unica', 'dose_total_pres_g', etc.

    Retorna: df_att (uma linha por cod_atendimento).
    '''
    if df.empty:
        return df.copy()

    out = df.copy()

    # Garantir datetime
    out['data_atendimento'] = pd.to_datetime(out['data_atendimento'], errors='coerce')

    # Criar features proxy se necessário
    needed = {'dias_tratamento', 'e_dose_unica', 'dose_total_pres_g'}
    if ensure_proxy_features and not needed.issubset(out.columns):
        # assume que add_ddd_proxy_features já está importada no seu projeto
        out = add_ddd_proxy_features(out)

    # Helper: agregações seguras para NaN
    def _nanmedian(x: pd.Series) -> float:
        x = pd.to_numeric(x, errors='coerce')
        return float(np.nanmedian(x.to_numpy())) if x.notna().any() else np.nan

    def _nanmean(x: pd.Series) -> float:
        x = pd.to_numeric(x, errors='coerce')
        return float(np.nanmean(x.to_numpy())) if x.notna().any() else np.nan

    def _nanmax(x: pd.Series) -> float:
        x = pd.to_numeric(x, errors='coerce')
        return float(np.nanmax(x.to_numpy())) if x.notna().any() else np.nan

    def _pct_true(x: pd.Series) -> float:
        # retorna percentual (0-100)
        if x is None or len(x) == 0:
            return np.nan
        x = x.astype('bool')
        return float(100.0 * x.mean())

    # Filtra prescrição ATB para métricas terapêuticas
    atb_mask = out['e_antibiotico'].fillna(0).astype(int).eq(1)
    out_atb = out.loc[atb_mask].copy()

    # Chaves/dimensões (pegar 1º valor por atendimento)
    dim_cols = [
        'cod_paciente',
        'data_atendimento',
        'sexo',
        'idade',
        'faixa_etaria',
        'especialidade',
        'cod_unidade_saude',
        'nome_unidade',
        'tipo',
        'ano',
        'mes',
        'nome_mes',
        'ano_mes',
        'diag_agrupado',
        'diag_analise',
        'diagnosticar_por',
    ]
    dim_cols = [c for c in dim_cols if c in out.columns]

    dims = (
        out.sort_values('data_atendimento')
        .groupby('cod_atendimento', as_index=False)[dim_cols]
        .first()
    )

    # Métricas básicas do atendimento
    # - presença de ATB / diagnóstico infeccioso
    base_metrics = out.groupby('cod_atendimento', as_index=False).agg(
        tem_atb=('e_antibiotico', lambda s: int(pd.to_numeric(s, errors='coerce').fillna(0).astype(int).gt(0).any())),
        tem_cid_infeccioso=('e_diag_infeccioso', lambda s: int(pd.to_numeric(s, errors='coerce').fillna(0).astype(int).gt(0).any())),
        n_medicacoes=('cod_medicamento', 'nunique'),
        n_cid=('cod_cid_ciap', lambda s: s.dropna().nunique()),
        e_presc_inadequada_att=('e_presc_inadequada', lambda s: int(pd.to_numeric(s, errors='coerce').fillna(0).astype(int).gt(0).any())),
    )

    # Métricas específicas de ATB (só no subconjunto com ATB)
    if out_atb.empty:
        atb_metrics = pd.DataFrame({'cod_atendimento': out['cod_atendimento'].dropna().unique()})
        atb_metrics['n_antibioticos'] = 0
        atb_metrics['dias_trat_medio'] = np.nan
        atb_metrics['dias_trat_mediana'] = np.nan
        atb_metrics['dias_trat_max'] = np.nan
        atb_metrics['pct_dose_unica'] = np.nan
        atb_metrics['dose_total_pres_g_sum'] = np.nan
    else:
        atb_metrics = out_atb.groupby('cod_atendimento', as_index=False).agg(
            n_antibioticos=('composto_quimico', lambda s: s.dropna().nunique()),
            dias_trat_medio=('dias_tratamento', _nanmean),
            dias_trat_mediana=('dias_tratamento', _nanmedian),
            dias_trat_max=('dias_tratamento', _nanmax),
            pct_dose_unica=('e_dose_unica', _pct_true),
            dose_total_pres_g_sum=('dose_total_pres_g', _nanmean),  # troque p/ sum se fizer sentido no seu caso
        )

    # Merge final
    df_att = (
        dims.merge(base_metrics, on='cod_atendimento', how='left')
            .merge(atb_metrics, on='cod_atendimento', how='left')
    )

    # defaults coerentes
    df_att['n_antibioticos'] = df_att['n_antibioticos'].fillna(0).astype(int)
    df_att['n_medicacoes'] = df_att['n_medicacoes'].fillna(0).astype(int)
    df_att['n_cid'] = df_att['n_cid'].fillna(0).astype(int)

    # se preferir dose_total como soma e não média, mude acima:
    # dose_total_pres_g_sum=('dose_total_pres_g','sum') e aqui manter float

    return df_att

In [19]:
df_att = build_attendance_level_df(df)
df_att

,cod_atendimento,cod_paciente,data_atendimento,sexo,idade,faixa_etaria,especialidade,cod_unidade_saude,nome_unidade,tipo,...,tem_cid_infeccioso,n_medicacoes,n_cid,e_presc_inadequada_att,n_antibioticos,dias_trat_medio,dias_trat_mediana,dias_trat_max,pct_dose_unica,dose_total_pres_g_sum
0,0000b4af921db374095a9e7ab87ee314522e4fe4,4708,2023-08-08,M,76,60+ anos,MÉDICO DA ESTRATÉGIA DE SAÚDE DA FAMÍLIA 225142,49,UBS NAIR SPINA BENEDICTIS,atenção primária,...,0,4,2,0,0,NaN,NaN,NaN,NaN,NaN
1,00019328d3a64a85b98cc0c18b03c1ca2044678a,20054,2023-06-05,F,60,60+ anos,MÉDICO DA ESTRATÉGIA DE SAÚDE DA FAMÍLIA 225142,46,UBS JOAO LUIZ PASQUAL BONAPARTE,atenção primária,...,0,3,3,0,0,NaN,NaN,NaN,NaN,NaN
2,0001bb7ae08980801982865a69fa8697d17e00c6,9608,2023-09-13,F,25,18-59 anos,MÉDICO DA ESTRATÉGIA DE SAÚDE DA FAMÍLIA 225142,17,CENTRO DE ESPECIALIDADES MEDICAS - CEM,ambulatorial,...,0,3,1,0,0,NaN,NaN,NaN,NaN,NaN
3,00020c775d4e68d28ef21441bdfb517c060964ae,54338,2023-07-03,F,48,18-59 anos,MÉDICO GINECOLOGISTA E OBSTETRA 225250,49,UBS NAIR SPINA BENEDICTIS,atenção primária,...,0,1,1,0,0,NaN,NaN,NaN,NaN,NaN
4,0002f28661e2bb706292acd5752f212f6dda0e23,26375,2023-05-19,F,71,60+ anos,MÉDICO DA ESTRATÉGIA DE SAÚDE DA FAMÍLIA 225142,43,UBS DOLORES MASSEI,atenção primária,...,0,5,1,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94239,fffa0f6a4222aa005f44ed40c3494b0bed1ef3f5,68446,2023-07-12,F,53,18-59 anos,CIRURGIÃO-DENTISTA DA ESTRATÉGIA DE SAÚDE DA F...,41,UBS CATERINA DALLANESE,atenção primária,...,0,1,1,0,0,NaN,NaN,NaN,NaN,NaN
94240,fffa77831abb5b07132f24bbfd13be397f2e1856,431,2023-04-04,M,71,60+ anos,MÉDICO DA ESTRATÉGIA DE SAÚDE DA FAMÍLIA 225142,41,UBS CATERINA DALLANESE,atenção primária,...,0,5,4,0,0,NaN,NaN,NaN,NaN,NaN
94241,fffb5483db5f64a52dbcdd381fde704b2812d224,31866,2023-06-21,F,33,18-59 anos,MÉDICO DA ESTRATÉGIA DE SAÚDE DA FAMÍLIA 225142,45,UBS DR IVANHOE ESPOSITO,atenção primária,...,0,1,1,0,0,NaN,NaN,NaN,NaN,NaN
94242,fffba8e8fec51f8d028e5c8570dbb9b5d21bd73a,73989,2023-08-28,F,78,60+ anos,MÉDICO DA ESTRATÉGIA DE SAÚDE DA FAMÍLIA 225142,41,UBS CATERINA DALLANESE,atenção primária,...,0,6,4,0,0,NaN,NaN,NaN,NaN,NaN


In [8]:
cols_novas = [
    'dias_tratamento',
    'vezes_ao_dia',
    'qtd_por_dose',
    'conc_g_por_unid',
    'dose_total_pres_g',
    'e_dose_unica'
]

df[cols_novas].head()

,dias_tratamento,vezes_ao_dia,qtd_por_dose,conc_g_por_unid,dose_total_pres_g,e_dose_unica
0,0.250000,4.0,1.0,0.50,0.50,True
1,5.000000,3.0,1.0,0.60,9.00,False
2,NaN,1.0,1.0,NaN,NaN,False
3,0.333333,3.0,1.0,0.05,0.05,True
4,7.000000,6.0,2.0,0.20,16.80,False


In [9]:
df[[
    'dias_tratamento',
    'vezes_ao_dia',
    'qtd_por_dose',
    'conc_g_por_unid',
    'dose_total_pres_g'
]].describe()

,dias_tratamento,vezes_ao_dia,qtd_por_dose,conc_g_por_unid,dose_total_pres_g
count,146330.000000,470536.000000,470536.000000,387628.000000,126709.000000
mean,6.819739,1.351238,2.185225,0.395579,23.172290
std,17.892025,0.818274,17.334613,23.911897,2810.113171
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.500000,1.000000,1.000000,0.020000,0.050000
50%,0.500000,1.000000,1.000000,0.040000,0.125000
75%,5.000000,1.000000,1.000000,0.100000,0.750000
max,505.041667,24.000000,5000.000000,5631.000000,473250.000000


In [11]:
df[cols_novas].isna().mean().sort_values(ascending=False)

dose_total_pres_g    0.730713
dias_tratamento      0.689014
conc_g_por_unid      0.176199
vezes_ao_dia         0.000000
qtd_por_dose         0.000000
e_dose_unica         0.000000
dtype: float64

In [12]:
df['dias_tratamento'].value_counts().sort_index().head(20)

dias_tratamento
0.000000      140
0.041667     1332
0.083333      455
0.125000      128
0.166667      677
0.208333        5
0.250000    20703
0.291667       52
0.333333    11166
0.375000       11
0.416667      100
0.458333       68
0.500000    48674
0.541667        3
0.583333      100
0.625000      266
0.666667       97
0.708333       43
0.750000       22
0.791667       47
Name: count, dtype: int64

In [13]:
df['vezes_ao_dia'].value_counts()

vezes_ao_dia
1.0     371893
2.0      57364
4.0      20889
3.0      19258
6.0        757
12.0       161
8.0        101
5.0         55
0.0         45
24.0        12
10.0         1
Name: count, dtype: int64

In [14]:
df['e_dose_unica'].value_counts(normalize=True)

e_dose_unica
False    0.803887
True     0.196113
Name: proportion, dtype: float64

In [15]:
df[
    ['duracao',
     'dias_tratamento',
     'vezes_ao_dia',
     'qtd_por_dose',
     'e_dose_unica']
].sample(20, random_state=42)

,duracao,dias_tratamento,vezes_ao_dia,qtd_por_dose,e_dose_unica
198887,TOMAR 01CP. DE 12/12H.,0.500000,2.0,1.0,True
43612,"TOMAR 01CP DE 12/12H, VIA ORAL, AO DIA.",0.500000,2.0,1.0,True
307279,TOMAR UM COMPRIMIDO DE 12/12H TODOS OS DIAS,0.500000,2.0,1.0,True
401408,TOMAR 1 COMPRIMIDO DE MANHA E 1 A NOITE,NaN,1.0,1.0,False
343825,TOMAR 2 CP DE MANHA,NaN,1.0,2.0,False
383974,TOMAR 01 COMPRIMIDO A NOITE.,NaN,1.0,1.0,False
322716,TOMAR 1CP PELA MANHA EM JEJUM.,NaN,1.0,1.0,False
338549,TOMAR 01 CP A NOITE,NaN,1.0,1.0,False
288741,TOMAR 2CP VO DE 12/12 HORAS,0.500000,2.0,2.0,True
58872,TOMAR 1 CP DE 8/8 HORAS,0.333333,3.0,1.0,True


In [16]:
df_atb = df[df['e_antibiotico'] == 1]

df_atb[['dias_tratamento', 'e_dose_unica']].describe()

,dias_tratamento
count,9868.000000
mean,7.591111
std,6.886991
min,0.041667
25%,7.000000
50%,7.000000
75%,7.000000
max,505.041667


In [20]:
df_att[['dias_trat_medio',
        'dias_trat_mediana',
        'pct_dose_unica',
        'n_antibioticos']].describe()

,dias_trat_medio,dias_trat_mediana,pct_dose_unica,n_antibioticos
count,6876.000000,6876.000000,7115.000000,94244.000000
mean,7.245299,7.245016,7.424455,0.082764
std,7.415666,7.419316,25.595660,0.301984
min,0.041667,0.041667,0.000000,0.000000
25%,5.000000,5.000000,0.000000,0.000000
50%,7.000000,7.000000,0.000000,0.000000
75%,7.000000,7.000000,0.000000,0.000000
max,505.041667,505.041667,100.000000,4.000000


In [21]:
df.sort_values('dias_tratamento', ascending=False)[
    ['duracao', 'dias_tratamento']
].head(20)

,duracao,dias_tratamento
301801,1 cp vo 12121 hs,505.041667
424207,"Tomar 02 comprimidos, via oral, 1x/dia MANTE...",252.000000
167048,TOMAR 1 CP POR SEMANA POR 9 MESES (36 SEMANAS),252.000000
88362,TOMAR 1 CP AO DIA ATÉ 36 SEMANAS DA GESTAÇÃO,252.000000
167051,TOMAR 1 CP POR SEMANA POR 9 MESES (36 SEMANAS),252.000000
167050,TOMAR 1 CP POR SEMANA POR 9 MESES (36 SEMANAS),252.000000
167049,TOMAR 1 CP POR SEMANA POR 9 MESES (36 SEMANAS),252.000000
388913,TOMAR 1 CP POR SEMANA POR 9 MESES (36 SEMANAS),252.000000
428702,TOMAR 1 CP POR SEMANA POR 9 MESES (36 SEMANAS),252.000000
321148,TOMAR 1 CP POR SEMANA POR 9 MESES (36 SEMANAS),252.000000


In [22]:
df_atb.groupby('composto_quimico')['dias_tratamento'].median().sort_values(ascending=False)

composto_quimico
CLARITHROMYCIN                    14.0
DOXYCYCLINE                       14.0
AMPICILLIN                        10.0
CLINDAMYCIN                        7.0
POLYMYXIN B                        7.0
NORFLOXACIN                        7.0
METRONIDAZOLE                      7.0
LEVOFLOXACIN                       7.0
GENTAMICIN                         7.0
AMOXICILLIN                        7.0
AMOXICILLIN+CLAVULANATE            7.0
CIPROFLOXACIN                      7.0
CEFUROXIME                         7.0
CEFALEXIN                          7.0
SULFAMETHOXAZOLE+TRIMETHOPRIME     7.0
AZITHROMYCIN                       5.0
CEFTRIAXONE                        3.0
BENZYLPENICILLIN/BENZATINE         1.0
Name: dias_tratamento, dtype: float64

In [23]:
df[['duracao','dias_tratamento','vezes_ao_dia']].head(15)

,duracao,dias_tratamento,vezes_ao_dia
0,Até 01 comprimido até de 6/6 h se dor (2 SE D...,0.250000,4.0
1,"TOMAR 01 COMP, VIA ORAL, DE 8/8H, POR 5 DIAS",5.000000,3.0
2,APLICAR SOBRE O LOCAL.,NaN,1.0
3,USO ORAL\n\nTOMAR 1 CP DE 8/8H SE DOR FORTE,0.333333,3.0
4,02 CP DE 4/4H POR 7 DIAS (5 TOMADAS AO DIA),7.000000,6.0
5,"TOMAR 01 COMP, VIA ORAL, DE 6/6H, POR 7 DIAS",7.000000,4.0
6,TOMAR 01 CP DE 12/12H,0.500000,2.0
7,TOMAR 01 CP DE 12/12 HORAS POR 5 DIAS,5.000000,2.0
8,TOMAR 01CP 1X AO DIA POR 5 DIAS,5.000000,1.0
9,TOMAR 01CP 1X À NOITE POR 7 DIAS,7.000000,1.0
